In [5]:
import os
import re

In [490]:
import pandas as pd

In [537]:
import numpy as np

In [394]:
from copy import copy

In [627]:
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [635]:
from sklearn.metrics import precision_score, recall_score, precision_recall_curve

Author: Аня Охапкина

# Разрешение анафоры и возвратные местоимения

Источник вдохновения: [Toldova, Ionov 2017]
    
http://www.dialog-21.ru/media/3956/toldovasionovm.pdf

В статье рассказывается об алгоритме для разрешения кореференции любых типов в РЯ. Я решила остановиться на русских возвратных местоимениях "себя" и "свой".

Благодаря Екатерине у меня уже был готовый корпус ru-coref, тот же самый, на базе которого написан алгоритм из статьи. В свой подкорпус я взяла только те предложения, в которых есть возвратные местоимения.

In [14]:
with open ('parsed_testset\\files_with_reflexives.txt', 'r', encoding='utf-8') as refl:
    refl_links = refl.read().split('\n')

In [32]:
refl_texts = []
for link in refl_links:
    with open (link.strip('.\\'), 'r', encoding='utf-8') as text:
        item = text.read()
        refl_texts.append(item)

In [33]:
self_sentences = []
for text in refl_texts:
    for item in text.split('\n\n'):
        if re.search('(себя)|(свой)', item, flags=re.DOTALL):
            self_sentences.append(item)

In [683]:
len(self_sentences)

293

Ура! У нас 293 предложения с рефлексивами!

Здесь вспомогательный кусок кода, в котором все предложения из self_sentences я записала в отдельный файл, чтобы вручную проставить антецеденты и связанные с ними возвратные местоимения.

In [360]:
# with open('self_sentences.txt', 'w', encoding='utf-8') as to_manual:
#     for sent in self_sentences:
#         to_manual.writelines(sent)
#         to_manual.write('\n\n')

Все предложения исходного корпуса обработаны в conll формате. 

Когда я размечала предложения руками, я добавила к каждому слову, которое считаю антецедентом чего-нибудь, помету 'ANTECEDENT' и порядковые номера слов, которые им связаны.

In [547]:
with open('self_sentences.txt', 'r', encoding='utf-8') as bound:
    self_sentences = bound.read().split('\n\n')

Так выглядят все пометы для слов в предложении:

In [548]:
nodes = []
for word in self_sentences[0].split('\n'):
    word = word.split('\t')
    #print(word)
    nodes.append([word[1], word[0], word[6], word[-2], word[-1]])

['1', 'Во', 'во', 'S', 'S', 'Sp-a', '9', 'обст', '_', '_']
['2', 'время', 'время', 'N', 'N', 'Ncnsan', '1', 'предл', '_', '_']
['3', 'своих', 'свой', 'P', 'P', 'P---pga', '4', 'опред', '_', '_']
['4', 'прогулок', 'прогулка', 'N', 'N', 'Ncfpgn', '2', '1-компл', '_', '_']
['5', 'в', 'в', 'S', 'S', 'Sp-l', '4', 'атриб', '_', '_']
['6', 'окрестностях', 'окрестность', 'N', 'N', 'Ncfpln', '5', 'предл', '_', '_']
['7', 'Симеиза', 'симеиз', 'N', 'N', 'Ncmsgn', '6', 'квазиагент', '_', '_']
['8', 'я', 'я', 'P', 'P', 'P-1-snn', '9', 'предик', '_', 'ANTECEDENT', '3']
['9', 'обратил', 'обратить', 'V', 'V', 'Vmis-sma-p', '0', 'ROOT', '_', '_']
['10', 'внимание', 'внимание', 'N', 'N', 'Ncnsan', '9', '1-компл', '_', '_']
['11', 'на', 'на', 'S', 'S', 'Sp-a', '10', '1-компл', '_', '_']
['12', 'одинокую', 'одинокий', 'A', 'A', 'Afpfsaf', '13', 'опред', '_', '_']
['13', 'дачу', 'дача', 'N', 'N', 'Ncfsan', '11', 'предл', '_', '_']
['14', ',', ',', ',', ',', ',', '13', 'PUNC', '_', '_']
['15', 'стоявшую', '

По этому списку я хотела строить дерево, но потом поняла, что обойдусь без него.

In [549]:
nodes

[['Во', '1', '9', '_', '_'],
 ['время', '2', '1', '_', '_'],
 ['своих', '3', '4', '_', '_'],
 ['прогулок', '4', '2', '_', '_'],
 ['в', '5', '4', '_', '_'],
 ['окрестностях', '6', '5', '_', '_'],
 ['Симеиза', '7', '6', '_', '_'],
 ['я', '8', '9', 'ANTECEDENT', '3'],
 ['обратил', '9', '0', '_', '_'],
 ['внимание', '10', '9', '_', '_'],
 ['на', '11', '10', '_', '_'],
 ['одинокую', '12', '13', '_', '_'],
 ['дачу', '13', '11', '_', '_'],
 [',', '14', '13', '_', '_'],
 ['стоявшую', '15', '13', '_', '_'],
 ['на', '16', '15', '_', '_'],
 ['крутом', '17', '18', '_', '_'],
 ['склоне', '18', '16', '_', '_'],
 ['горы', '19', '18', '_', '_'],
 ['.', '20', '19', '_', '_']]

Дальше я преобразовала сырые данные из корпуса в формат, пригодный для обучения.

Я выбрала 8 параметров для своего пробного классификатора: 

1. Потенциальный антецедент с-командует анафором (анафор зависит от того же, от чего антецедент, или от кого-то из его потомков)

2. Потенциальный антецедент скорее в теме, чем в реме (попробуем просто смотреть на линейный порядок, как будто до главного глагола (привязка == 0) идёт тема, а после рема: номер антецедента меньше номера глагола, от которого зависит анафор)

3. Глагол согласуется по числу с потенциальным антецедентом (помета числа глагола, от которого зависит местоимение, совпадает с пометой числа антецедента)

4. Потенциальный антецедент -- местоимение (помета в тегах соответствует местоимению)

5. Анафор зависит от антецедента (номер родителя анафора == номер антецедента)

6. Потенциальный антецедент в номинативе (соответствующая помета в тегах == n)

7. Потенциальный антецедент в линеаризации идёт до анафора (номер антецедента меньше номера анафора)

8. Потенциальный антецедент согласуется с анафором по числу и падежу (например, "своей кошки"). Это фактор, который работает наоборот: такие пары не связаны.

In [685]:
class SentenceProcessing:
    def __init__(self, sentence):
        self.info = []
        for word in sentence.split('\n'):
            word = word.split('\t')
            self.info.append({'number': int(word[0]), 'word': word[1], 'tags': word[5], 'alias': int(word[6]),
                              #'is_antecedent': word[-2], 
                              'bindee': word[-1]})

            
    def preprocess(self):
        self.verblist = []
        self.antecedents = []
        self.anaphors = []
        self.tree = {}
        for word in self.info:
            
            if word['tags'].startswith('V'):
                self.verblist.append(word)

            if word['tags'].startswith('N') or re.search('P-[123]', word['tags']):
                self.antecedents.append(word)
                
            if re.match('(P----.n)|(P---..a)', word['tags']) and word['tags'] != 'P-----a':
                self.anaphors.append(word)
        
            if word['alias'] not in self.tree:
                self.tree[word['alias']] = [word['number']]
            else:
                self.tree[word['alias']].append(word['number'])
        
    
    def antecedent_features(self, antecedent):
        answer_2 = 0
        answer_3 = 0
        answer_4 = 0
        answer_6 = 0
        for verb in self.verblist:
            if verb['alias']==0:
                
                if antecedent['number'] < verb['number']:
                    answer_2 = 1
                
                if antecedent['tags'][3] == verb['tags'][5]:
                    answer_3 = 1
        
        if antecedent['tags'].startswith('P'):
            answer_4 = 1
        if antecedent['tags'][-2] == 'n':
            answer_6 = 1
        
        return [answer_2, answer_3, answer_4, answer_6]

    
    def antecedent_anaphor_features(self, antecedent, anaphor):
        answer_5 = 0
        answer_7 = 0
        answer_8 = 0
        
        if antecedent['number'] < anaphor['number']:
            answer_7 = 1
        
        if anaphor['alias'] == antecedent['number']:
            answer_5 = 1
        
        if anaphor['tags'][-3:-1] == antecedent['tags'][-3:-1]:
            answer_8 = 1
        
        return [answer_5, answer_7, answer_8]
        
        
    def c_command(self, antecedent, anaphor, head):
        #print(antecedent) # смотрим, с каким антецедентом мы работаем
        answer_1 = 0 # по умолчанию параметр с-командования равен 0
        if head in self.tree:
            kids = self.tree[head] # найдём всех сестер антецедента
            #print(kids)
            for node in kids: # для всех сестёр антецедента делаем вот что:
                #print('checking now node ', node)
                if node == anaphor['number']: # проверяем, что анафор не сестра антецедента
                    #print('found it!')
                    answer_1 = 1
                    #print ('answer_1 = ', answer_1)
                    break
                if node not in self.tree:
                    continue
                if node != antecedent['number']:
                    answer_1 = self.c_command(antecedent, anaphor, node)
                    if answer_1:
                        break
        
        #print(answer_1)
        return answer_1
    
    def processing(self):
        for_table = []
        ant_alias = None
        for antecedent in self.antecedents:
            for anaphor in self.anaphors:
                [answer_2, answer_3, answer_4, answer_6] = self.antecedent_features(antecedent)
                [answer_5, answer_7, answer_8] = self.antecedent_anaphor_features(antecedent, anaphor)
                answer_1 = self.c_command(antecedent, anaphor, antecedent['alias'])
                #print(str(anaphor['number']) in antecedent['bindee'].split(','))
                if str(anaphor['number']) in antecedent['bindee'].split(','):
                    antecedent['is_antecedent'] = 1
                else:
                    antecedent['is_antecedent'] = 2
                #print(antecedent['is_antecedent'])
                for_table.append([(antecedent['word'], anaphor['word']), (antecedent['number'], anaphor['number']),
                                  answer_1, answer_2, answer_3, answer_4, 
                                  answer_5, answer_6, answer_7, answer_8,
                                  antecedent['is_antecedent'], antecedent['bindee']])
        return copy(for_table)

In [713]:
for_df = []
num = 1
for sentence in (self_sentences):
    if sentence:
        sent = SentenceProcessing(sentence)
        sent.preprocess()
        list_of_features=sent.processing()
        for j, item in enumerate(list_of_features):
            for i, feature in enumerate(item):
                if isinstance(feature, tuple):
                    item[i] = ' -> '.join([str(part) for part in feature])
            list_of_features[j] = [num] + item
            #print(item)
    for_df += list_of_features
    num += 1

In [714]:
base_df = pd.DataFrame(for_df, columns=['sentence_num', 'pair', 'pair_nums', 'c-command', 'theme', 'verb_agree', 'pronoun',
                                        'parent', 'nominative', 'antecedent_earlier', 'ant_anaph_agree', 
                                        'is_antecedent', 'bindee'])

Вот так выглядит таблица, понятная человеку:

In [715]:
base_df.head(25)

,sentence_num,pair,pair_nums,c-command,theme,verb_agree,pronoun,parent,nominative,antecedent_earlier,ant_anaph_agree,is_antecedent,bindee
0,1,время -> своих,2 -> 3,0,1,1,0,0,0,1,0,2,_
1,1,прогулок -> своих,4 -> 3,0,1,0,0,1,0,0,1,2,_
2,1,окрестностях -> своих,6 -> 3,0,1,0,0,0,0,0,0,2,_
3,1,Симеиза -> своих,7 -> 3,0,1,1,0,0,0,0,0,2,_
4,1,я -> своих,8 -> 3,1,1,0,1,0,1,0,0,1,3
5,1,внимание -> своих,10 -> 3,1,0,1,0,0,0,0,0,2,_
6,1,дачу -> своих,13 -> 3,0,0,1,0,0,0,0,0,2,_
7,1,склоне -> своих,18 -> 3,0,0,1,0,0,0,0,0,2,_
8,1,горы -> своих,19 -> 3,0,0,1,0,0,0,0,0,2,_
9,2,Я -> своих,1 -> 6,1,1,0,1,0,1,1,0,1,6


Вытаскиваем из неё куски, понятные машине

In [716]:
first_data = copy(base_df)
first_data.drop(['sentence_num', 'pair_nums', 'pair', 'bindee'], axis=1, inplace=True)
X = copy(first_data)
X.drop('is_antecedent', axis=1, inplace=True)
y = first_data['is_antecedent']

In [717]:
len(first_data)

1812

У меня в датасете оказалось 1812 потенциальных антецедентов для разных местоимений.

In [718]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [719]:
best_score = 0
best_params = (None, None)
for n_trees in range(1, 15):
    for max_depth in range(3, 25):
        cl = RandomForestClassifier(n_estimators=n_trees, max_depth=max_depth)
        scores = cross_validate(cl, X_train, y_train, cv=3, scoring='f1')['test_score']
        current_score = sum(scores) / len(scores)
        if current_score > best_score:
            best_score = current_score
            best_params = (n_trees, max_depth)
print(best_score, best_params)

C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: Unde

C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1143: Unde

0.5062557294136242 (2, 7)


Предупреждение, выводящееся на экран, означает, что среди y_validate есть значения, ни разу не встретившиеся в predicted. То есть мой классификатор, скорее всего, слишком часто предсказывает, что слово не является антецедентом :(

In [730]:
clf = RandomForestClassifier(n_estimators=best_params[0], max_depth=best_params[1])
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
precision_new = precision_score(y_pred=predicted, y_true=y_test)
recall_new = recall_score(y_pred=predicted, y_true=y_test)
f1_new = f1_score(y_pred=predicted, y_true=y_test)

In [731]:
print(precision_new, recall_new, f1_new, sep='\n')

0.5
0.40625
0.4482758620689655


## главная ошибка

Кажется, что f-мера падает из-за большого количества true negative векторов (много именных групп и мало анафоров в предложениях):

In [681]:
dummy_anteceds = pd.get_dummies(first_data['is_antecedent'])
for column in dummy_anteceds.columns:
    print(column, ':   ', len(dummy_anteceds[dummy_anteceds[column]==1]))

1 :    137
2 :    1675


## другие ошибки

Не лучшая имплементация проверки с-командования (на дереве составляющих было бы лучше, но мне на это не хватило времени). Зато это реализовано в коде, на статью о котором я опиралась.

Неудачное разбиение самих текстов корпуса на предложения. Если посмотреть на файлы корпуса, то можно заметить, что разбивка на предложения далека от идеала. Я старалась убирать из выборки совсем неудачно выделенные предложения, но часть оставляла.

Интуитивно непонятный алгоритм построения зависимостей. Если в начале работы я предполагала, что корень дерева -- всегда главный глагол, потом оказалось, что это не так.

## что внутри класса

Как выглядят атрибуты:

In [431]:
sent = SentenceProcessing(self_sentences[0])

In [432]:
sent.preprocess()

In [433]:
list_of_features = sent.processing()

{'number': 2, 'word': 'время', 'tags': 'Ncnsan', 'alias': 1, 'is_antecedent': '_', 'bindee': '_'}
[2]
checking now node  2
0
{'number': 4, 'word': 'прогулок', 'tags': 'Ncfpgn', 'alias': 2, 'is_antecedent': '_', 'bindee': '_'}
[4]
checking now node  4
0
{'number': 6, 'word': 'окрестностях', 'tags': 'Ncfpln', 'alias': 5, 'is_antecedent': '_', 'bindee': '_'}
[6]
checking now node  6
0
{'number': 7, 'word': 'Симеиза', 'tags': 'Ncmsgn', 'alias': 6, 'is_antecedent': '_', 'bindee': '_'}
[7]
checking now node  7
0
{'number': 8, 'word': 'я', 'tags': 'P-1-snn', 'alias': 9, 'is_antecedent': 'ANTECEDENT', 'bindee': '3'}
[1, 8, 10]
checking now node  1
{'number': 8, 'word': 'я', 'tags': 'P-1-snn', 'alias': 9, 'is_antecedent': 'ANTECEDENT', 'bindee': '3'}
[2]
checking now node  2
{'number': 8, 'word': 'я', 'tags': 'P-1-snn', 'alias': 9, 'is_antecedent': 'ANTECEDENT', 'bindee': '3'}
[4]
checking now node  4
{'number': 8, 'word': 'я', 'tags': 'P-1-snn', 'alias': 9, 'is_antecedent': 'ANTECEDENT', 'bind

In [434]:
for item in list_of_features:
    for i, feature in enumerate(item):
        if isinstance(feature, tuple):
            item[i] = ' -> '.join(feature)

In [435]:
for item in list_of_features:
    for feature in item:
        print(feature, type(feature))

время -> своих <class 'str'>
0 <class 'int'>
1 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
_ <class 'str'>
_ <class 'str'>
прогулок -> своих <class 'str'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
1 <class 'int'>
_ <class 'str'>
_ <class 'str'>
окрестностях -> своих <class 'str'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
_ <class 'str'>
_ <class 'str'>
Симеиза -> своих <class 'str'>
0 <class 'int'>
1 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
0 <class 'int'>
_ <class 'str'>
_ <class 'str'>
я -> своих <class 'str'>
1 <class 'int'>
1 <class 'int'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
1 <class 'int'>
0 <class 'int'>
0 <class 'int'>
ANTECEDENT <class 'str'>
3 <class 'str'>
внимание -> своих <class 'str'>
1 <cla

In [73]:
sent.antecedents

[{'number': '2', 'word': 'время', 'tags': 'Ncnsan', 'alias': '1'},
 {'number': '4', 'word': 'прогулок', 'tags': 'Ncfpgn', 'alias': '2'},
 {'number': '6', 'word': 'окрестностях', 'tags': 'Ncfpln', 'alias': '5'},
 {'number': '7', 'word': 'Симеиза', 'tags': 'Ncmsgn', 'alias': '6'},
 {'number': '8', 'word': 'я', 'tags': 'P-1-snn', 'alias': '9'},
 {'number': '10', 'word': 'внимание', 'tags': 'Ncnsan', 'alias': '9'},
 {'number': '13', 'word': 'дачу', 'tags': 'Ncfsan', 'alias': '11'},
 {'number': '18', 'word': 'склоне', 'tags': 'Ncmsln', 'alias': '16'},
 {'number': '19', 'word': 'горы', 'tags': 'Ncfsgn', 'alias': '18'}]

In [74]:
sent.verblist

[{'number': '9', 'word': 'обратил', 'tags': 'Vmis-sma-p', 'alias': '0'},
 {'number': '15', 'word': 'стоявшую', 'tags': 'Vmps-sfa-ea', 'alias': '13'}]

In [75]:
sent.anaphors

[{'number': '3', 'word': 'своих', 'tags': 'P---pga', 'alias': '4'}]

In [176]:
sent.tree

{'9': ['1', '8', '10'],
 '1': ['2'],
 '4': ['3', '5'],
 '2': ['4'],
 '5': ['6'],
 '6': ['7'],
 '0': ['9'],
 '10': ['11'],
 '13': ['12', '14', '15'],
 '11': ['13'],
 '15': ['16'],
 '18': ['17', '19'],
 '16': ['18'],
 '19': ['20']}